In [ ]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [ ]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT * 
FROM Z3DMC.MEB_DGM
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
#Po wykonaniu LEFT JOIN liczba rekordów się zwiększyła, trzeba usunąć duplikaty z DMC_CASTING ~ LO
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY')
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
select id_dmc, circuit_nr, 
max(assigment) as assigment, 
max(flow) as flow, 
max(ref_time) as ref_time,
max(set_point) as set_point,
max(start_delay) as start_delay,
max(temp) as temp,
max(working_mode) as working_mode,
max(timestamp) as timestamp
from z3dmc.oni_circuits
group by id_dmc, circuit_nr
order by id_dmc
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [ ]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [ ]:
print(list(data.keys()))
# Useless tables
# MEB_CONTAINERS, MEB_DMC_GRADE, MEB_KS_WZORC

In [ ]:
# ONI CIRCUITS
# print(data['ONI_CIRCUITS'].head())
data['ONI_CIRCUITS'].drop(columns=['timestamp'], inplace=True)
print("\n ONI_CIRCUITS after drop of columns: \n")
print(data['ONI_CIRCUITS'].head())

In [ ]:
# MEB_DGM
# print(data['MEB_DGM'].columns)
# print(data['MEB_DGM'].head(5))
# ('metal_level', 'metal_pressure') - We do not collect this data yet
data['MEB_DGM'].drop(columns=['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'], inplace=True)
print("\n MEB_DGM after drop of columns: \n")
print(data['MEB_DGM'].head(5))

In [ ]:
#MEB_DMC
# print(data['MEB_DMC'].columns)
# print(data['MEB_DMC'].head(5))
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step'], inplace = True)
print("\n MEB_DMC after drop of columns: \n")
print(data['MEB_DMC'].head(5))

In [ ]:
#MEB_KO
# print(data['MEB_KO'].columns)
# print(data['MEB_KO'].head(5))
data['MEB_KO'].drop(columns = ['id_ko', 'data', 'timestamp', 'eks'], inplace = True)
print("\n MEB_KO after drop of columns: \n")
print(data['MEB_KO'].head(5))

In [ ]:
#MEB_KO_DGM
# print(data['MEB_KO_DGM'].columns)
# print(data['MEB_KO_DGM'].head(5))
data['MEB_KO_DGM'].drop(columns = ['id_ko','data_odlania', 'timestamp', 'operator'], inplace = True)
print("\n MEB_KO_DGM after drop of columns: \n")
print(data['MEB_KO_DGM'].head(5))

In [ ]:
# MEB_KS
# print(data['MEB_KS'].columns)
# print(data['MEB_KS'].head(5))
data['MEB_KS'].drop(columns = ['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny'], inplace = True)
print("\n 'MEB_KS' after drop of columns: \n")
print(data['MEB_KS'].head(5))

In [ ]:
#MEB_GROB 
# print(data['MEB_GROB'].columns)
# print(data['MEB_GROB'].head(5))
data['MEB_GROB'].drop(columns = ['shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested', 
                                'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'], inplace = True)
print("\n 'MEB_GROB' after drop of columns: \n")
print(data['MEB_GROB'].head(5))


<h1>Łączenie tabel</h1>

In [ ]:
final_table = data['MEB_DMC'].copy()

In [ ]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)


#data['MEB_KO'].head()
#data['MEB_KO_DGM'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KO
print(final_table.shape)
final_table = final_table.merge(data['MEB_KO'], on='id_dmc', how='left')
print(final_table.shape)

In [ ]:
final_table.drop(columns=['rn'], inplace=True)

In [ ]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
print(data['MEB_DGM'].shape)
data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
print(data['MEB_DGM'].shape)


In [ ]:
data['MEB_DGM'].drop(columns=['rn'], inplace=True)

In [ ]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 

oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'working_mode', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM
print(f"Before meb_dgm to oni circuits{len(oni_circuits)}")

oni_circuits = oni_circuits.merge(data['MEB_DGM'], left_on='id_dmc', right_on='id', how='left')
#oni_circuits.head(1)
print(f"After meb_dgm to oni circuits{len(oni_circuits)}")

In [ ]:

print(list(oni_circuits.keys()))

filtered_values = [value for value in list(oni_circuits.keys()) if value.startswith('id')]

print(filtered_values)

print(oni_circuits[['id_dmc_x', 'id', 'id_dmc_y']].head(5))


In [ ]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS
print(f"Before oni circuits to meb_dmc{len(final_table)}")
final_table = final_table.merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left')
#data['MEB_DMC'].head(1)
print(f"after oni circuits to meb_dmc{len(final_table)}")

In [ ]:
data['MEB_DMC']['dmc_casting'] = data['MEB_DMC']['dmc_casting'].str.strip() 
oni_circuits['dmc'] = oni_circuits['dmc'].str.strip()  

data['MEB_DMC'].drop_duplicates(subset=['dmc_casting'], inplace=True)
oni_circuits.drop_duplicates(subset=['dmc'], inplace=True)

inner_j = data['MEB_DMC'].merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='inner')
left_j = data['MEB_DMC'].merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left')
print(f"MEB_DMC: {data['MEB_DMC'].shape}")
print(f"oni circuits: {oni_circuits.shape}")
print(f'inner: {inner_j.shape}')
print(f"left: {left_j.shape}")
print(f"final: {final_table.shape}")

In [ ]:
duplicate_count = data['MEB_DMC']['dmc_casting'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'MEB_DMC' value: {duplicate_count}")

In [ ]:
duplicate_count_oni = oni_circuits['dmc'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB
print(final_table.shape)
final_table = final_table.merge(data['MEB_GROB'], on='id_dmc', how='left')
print(final_table.shape)

In [ ]:
final_table.drop(columns=['rn'], inplace=True)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS
print(final_table.shape)
final_table = final_table.merge(data['MEB_KS'], on='id_dmc', how='left')
print(final_table.shape)
#data['MEB_DMC'].head(1)

In [ ]:
final_table.drop(columns=['rn'], inplace=True)

In [ ]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

In [ ]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

<h1>Usuwanie anomalii</h1>

In [ ]:
print(list(data.keys()))

In [ ]:
meb_dgm_df = data['MEB_DGM']

In [ ]:
print(meb_dgm_df.head(5))

In [ ]:
print(list(meb_dgm_df.keys()))

In [ ]:
print(meb_dgm_df['nr_dgm'].unique())
print(len(meb_dgm_df))
print(type(meb_dgm_df['nr_dgm'][0]))
print(len(meb_dgm_df[meb_dgm_df['nr_dgm'] == 20]))



In [ ]:
print(final_table.head(4))


In [ ]:
for i in final_table.keys():
    print(i)

In [ ]:
result = meb_dgm_df.groupby(final_table['dmc_x'].str[:3]).size().reset_index(name='count_of_results')
result.columns = ['unique_starting_letters', 'count_of_results']
result = result.sort_values(by='unique_starting_letters')

print(result)